In [1]:
from IPython.core.debugger import set_trace

from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
import pydot_ng as pydot

import sys
sys.path.append('../financial_utils/')

import keras
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

import performance as per



C:\Users\Jon\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# DATASET

### Load and Compute Returns

In [2]:
price_table = pd.read_csv('history_files/SPY.csv')
vol_table = pd.read_csv('history_files/VIX.csv')

# Build Tick Table
tick_table = price_table[["Date","AdjClose"]]
tick_table.columns = ["Date", "Tick"]
# Get Return Table
returns = per.tick2ret(tick_table)
returns_table = pd.DataFrame({'Date':tick_table.Date[1:], 'Return': returns[:,0]})
returns_table = returns_table.set_index('Date')
# Drop columns of Vol Table
vol_table = vol_table[["Date", "Close"]]
vol_table.columns = ["Date", "Vol"]
vol_table = vol_table.set_index('Date')
# InnerJoin
retvol_table = pd.concat([returns_table,vol_table], join='inner', axis=1)


In [3]:
T_x = 14
x_width = retvol_table.shape[1]
T_y = 1
T_stride = 1

### Generate Dataset

In [4]:
def gen_dataset(T_x, T_y, T_stride, retvol_table):
    """
    
    
    return
        x E (m, T_x)
        y E (m, T_y)
    """
    
    m = int(np.floor((returns.shape[0] - T_x) / T_stride))
    
    x = np.nan * np.ones((m, T_x, retvol_table.shape[1]))
    y = np.nan * np.ones((m, T_y))
    
    for i in range(m):
        x[i,:,0] = retvol_table.Return[i*T_stride:i*T_stride+T_x].transpose()
        x[i,:,1] = retvol_table.Vol[i*T_stride:i*T_stride+T_x].transpose()
        y[i] = retvol_table.Return[i*T_stride:i*T_stride+T_y].transpose()

    return x, y, m

In [5]:
x, y, m = gen_dataset(T_x,T_y,T_stride,retvol_table)
print('Training Example: '+str(m))
print('X Shape: '+str(x.shape))
print('Y Shape: '+str(y.shape))

Training Example: 6081
X Shape: (6081, 14, 2)
Y Shape: (6081, 1)


### Train / Val / Test

In [6]:
train_end = int(m * 0.7)

val_end = train_end + int(0.15 * m)

x_train = x[0:train_end]
x_val = x[train_end:val_end]
x_test = x[val_end:]

y_train = y[0:train_end]
y_val = y[train_end:val_end]
y_test = y[val_end:]

# Keras Model

In [34]:
keras.backend.clear_session()
n_a = 10
gru1 = keras.layers.GRU(units = n_a, return_sequences=True)
flatter = keras.layers.Flatten()
dense_out = keras.layers.Dense(units = 1, activation='relu')

In [35]:
def define_model(T_x, x_width, T_y, n_a):
    
    x_input = keras.layers.Input(shape=(T_x,x_width))
    s = x_input
    outputs = list()
    
    for t in range(T_y):
        s = gru1(s)
        s_flat = flatter(s)
        out = dense_out(s_flat)
        
        outputs.append(out)
    
    
    model = keras.models.Model(inputs=x_input, outputs=outputs)
    return model

In [36]:
model = define_model(T_x, x_width, T_y, n_a)

In [37]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 14, 2)             0         
_________________________________________________________________
gru_1 (GRU)                  (None, 14, 10)            390       
_________________________________________________________________
flatten_1 (Flatten)          (None, 140)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 141       
Total params: 531
Trainable params: 531
Non-trainable params: 0
_________________________________________________________________


In [38]:
optim = keras.optimizers.Adam()
model.compile(optimizer=optim, loss='mse', metrics=['accuracy'])

In [39]:
s0 = np.zeros((m,n_a))
c0 = np.zeros((m,n_a))
model.fit(x=x_train, y=y_train, shuffle=True, epochs = 10)

Epoch 1/10
4256/4256 [==============================] - 2s 560us/step - loss: 1.5718e-04 - acc: 0.0117
Epoch 2/10
4256/4256 [==============================] - 1s 346us/step - loss: 1.5718e-04 - acc: 0.0117
Epoch 3/10
4256/4256 [==============================] - 1s 347us/step - loss: 1.5718e-04 - acc: 0.0117
Epoch 4/10
4256/4256 [==============================] - 1s 349us/step - loss: 1.5718e-04 - acc: 0.0117
Epoch 5/10
4256/4256 [==============================] - 1s 346us/step - loss: 1.5718e-04 - acc: 0.0117
Epoch 6/10
4256/4256 [==============================] - 1s 347us/step - loss: 1.5718e-04 - acc: 0.0117
Epoch 7/10
4256/4256 [==============================] - 1s 347us/step - loss: 1.5718e-04 - acc: 0.0117
Epoch 8/10
4256/4256 [==============================] - 1s 345us/step - loss: 1.5718e-04 - acc: 0.0117
Epoch 9/10
4256/4256 [==============================] - 1s 347us/step - loss: 1.5718e-04 - acc: 0.0117
Epoch 10/10
4256/4256 [==============================] - 1s 346us/step - 

In [43]:
model.evaluate(x=x_test, y=y_test)

913/913 [==============================] - 0s 77us/step


[6.435496008439553e-05, 0.002190580503833516]

In [44]:
model.predict(x_test[1:50,:,:])

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)